In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json




def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds


def lambda_handler(event, context):
    # TODO implement
    print(event, context)

    creds = oauth()
    service = build('sheets', 'v4', credentials=creds)
    new_spreadsheet_id = event["spreadsheet_id"]
    new_daebong_spreadsheet_title = event["title"]
    try:
        # 수식 적용
        # apply_formula_to_cells(service, new_spreadsheet_id, '발주서', new_daebong_spreadsheet_title)
        # print("Formula applied to cells in the new spreadsheet. 발주서")

        # apply_formula_to_cell_sheet2(service, new_spreadsheet_id, '누적발주', new_daebong_spreadsheet_title)
        # print("Formula applied to cells in the new spreadsheet. 누적발주")

        # apply_protect_data(service, new_spreadsheet_id)
        print("Formula applied to cells in the new spreadsheet.")


    except HttpError as error:
        # HttpError에서 응답 코드와 메시지를 추출합니다.
        error_code = error.resp.status
        try:
            # 오류 응답 본문을 JSON 객체로 파싱합니다.
            error_details = json.loads(error.content.decode())
            error_message = error_details.get('error', {}).get('message', 'Unknown error')
        except json.JSONDecodeError:
            # 오류 응답 본문이 JSON이 아닌 경우, 일반 텍스트로 처리합니다.
            error_message = error.content.decode()

        print(f"An error occurred: {error}")
        return {
            'statusCode': error_code,  # HTTP 상태 코드를 응답에 설정합니다.
            'body': json.dumps({"error": error_message})  # 오류 메시지를 JSON 응답에 포함시킵니다.
        }

    return {
        'statusCode': 200,
        'body': json.dumps({"result": "Success", "message": "Formula applied successfully"})
    }


In [ ]:
def edit_user_info(service, spreadsheet_id, sheet_title, new_spreadsheet_id, title):
    range = f"{sheet_title}!D:E"  # D와 E열을 참조하는 범위
    
    try:
        # 지정된 범위에서 데이터를 읽어옵니다.
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range,
            majorDimension='ROWS'
        ).execute()
        
        values = result.get('values', [])
        
        # D열과 E열에서 첫 번째 빈 셀의 행 번호를 찾습니다.
        # values 리스트에는 D와 E열의 값이 행별로 저장되어 있습니다.
        last_row = len(values) + 1  # +1은 스프레드시트의 행이 1부터 시작하기 때문입니다.
        
        # 추가할 데이터를 설정합니다.
        update_values = [
            [title, f"https://docs.google.com/spreadsheets/d/{new_spreadsheet_id}/edit#gid=0"]
        ]
        
        # 지정된 위치에 데이터를 쓰기 위한 요청 본문을 준비합니다.
        body = {
            'values': update_values
        }
        
        # 스프레드시트에 데이터를 쓰기 위한 API 요청을 실행합니다.
        update_range = f"{sheet_title}!D{last_row}:E{last_row}"
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=update_range,
            valueInputOption='USER_ENTERED',
            body=body
        ).execute()
        
        print(f"Updated the spreadsheet with title: {title} and link: https://docs.google.com/spreadsheets/d/{new_spreadsheet_id}/edit#gid=0")
        
    except HttpError as err:
        print(err)